In [16]:
import pandas as pd
from langchain.chains import LLMChain, SequentialChain


In [17]:
import prompts as p
from langchain.prompts import PromptTemplate


PRODUCTS_AMAZON = '../../datasets/Amazon.csv'

PRODUCTS_GOOGLE = '../../datasets/GoogleProducts.csv'

PRODUCT_MAPPING = '../../datasets/GoogleAmazonSampled_400.csv'

SAVED_PROMPTS_PATH = '../../evals/saved_prompts_GA.csv'

# Load data
df_A = pd.read_csv(PRODUCTS_AMAZON, encoding='latin-1')
df_B = pd.read_csv(PRODUCTS_GOOGLE, encoding='latin-1')
df_mapping = pd.read_csv(PRODUCT_MAPPING)


df_A.fillna('', inplace=True)
df_B.fillna('', inplace=True)
df_mapping.fillna('', inplace=True)

prompt_name = 'prompt1_en_short_temp_0_gpt3.5-0301'
prompt = p.prompt1_en_short

MODEL = "gpt-3.5-turbo-0301"

MAX_TOKENS = 15


In [18]:
# generate column with prompt_name if not exists
if 'prompt_name' not in df_mapping.columns:
    df_mapping[prompt_name] = ""

In [19]:
df_mapping.head(30)

,Unnamed: 0,ltable_id,rtable_id,label,prompt1_en_short_temp_0_gpt4-0613,match_prompt_manual_fewshot_1_eval2_temp_0_gpt3.5_0301,prompt1_en_short_temp_0_gpt3.5-0301
0,0,363,2854,True,True,True,
1,1,1269,1433,True,True,True,
2,2,63,2129,True,False,False,
3,3,540,5,True,True,True,
4,4,1262,192,True,True,True,
5,5,1320,685,True,True,False,
6,6,1092,978,True,True,True,
7,7,1214,549,True,True,True,
8,8,1026,2327,True,True,False,
9,9,594,606,True,True,True,


In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI


In [21]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [22]:
from langfuse.callback import CallbackHandler
handler = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))

In [23]:
def cut_length(string ,length = 250):
    return string[0:length]

In [24]:

# product1 = "Title: " + cut_length(str(df["title_left"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][1] or "none"))
# product2 = "Title: " + cut_length(str(df["title_right"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][1] or "none"))



# product1 = "Title: " + cut_length(str(df["title_left"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][1] or "none")) + "\nBrand:" + str(df["brand_left"][1] or "none") 
# product2 = "Title: " + cut_length(str(df["title_right"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][1] or "none")) + "\nBrand:" + str(df["brand_right"][1] or "none") 

In [25]:
saved_prompts_df = pd.read_csv(SAVED_PROMPTS_PATH)

def saveResponse(prompt, response, isMatch, index):
    # create new column if not exists 
    if "prompt_" + prompt_name not in saved_prompts_df.columns:
        saved_prompts_df["prompt_" + prompt_name] = ""
        saved_prompts_df["response_" + prompt_name] = ""
        saved_prompts_df["isMatch_" + prompt_name] = ""


    
    # save response prompt and isMatch to df at index
    saved_prompts_df.at[index, "prompt_" + prompt_name] = prompt
    saved_prompts_df.at[index, "response_" + prompt_name] = response
    saved_prompts_df.at[index, "isMatch_" + prompt_name] = isMatch
    
    

In [26]:
def evaluate(df, prompt, temp=0, start = 0):
    # start = len(df) - start - 1
    if (MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name=MODEL, temperature=temp,max_tokens=MAX_TOKENS)
        print("Using OpenAI")
    else:
        llm = ChatOpenAI(model_name=MODEL, temperature=temp,max_tokens=MAX_TOKENS)
    for i in range(start, len(df)):
        print(i)
        # Creating Prompt and sending request and printing informations
        brandl = ""
        brandr = ""
        # if (df_A["manufacturer"][df["ltable_id"][i]]):
        #     brandl = "\nBrand: " + str(df_A["manufacturer"][df["ltable_id"][i]])
        # if (df_A["manufacturer"][df["rtable_id"][i]]):
        #     brandr = "\nBrand: " + str(df_A["brmanufacturerand"][df["rtable_id"][i]])

        product1 = "Title: " + cut_length(str(df_A["title"][df["ltable_id"][i]] or "none")) + brandl + "\nDescription: " + cut_length(str(df_A["description"][df["ltable_id"][i]] or "none"))
        product2 = "Title: " + cut_length(str(df_B["title"][df["rtable_id"][i]] or "none")) + brandr + "\nDescription: " + cut_length(str(df_B["description"][df["rtable_id"][i]] or "none"))
        
        chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
        
        try:
            response = chain.run(p1=product1, p2=product2, callbacks=[handler])        
        except Exception as e:
            print(e)
            i -= 1
            continue

        formatted_prompt = chain.prompt.format_prompt(p1=product1, p2=product2).text
        if(i < 5):
            print(formatted_prompt)
            print("__\n__")
            print(response)
            print("__\n__")
        
        #Processing Response    
        isMatch = False
        if ("True" in response or "true" in response or "yes" in response):
            isMatch = True
        elif ("False" in response or "false" in response or "no" in response):
            isMatch = False
        
        df[prompt_name][i] = isMatch


        saveResponse(formatted_prompt, response, isMatch, i)

In [27]:
evaluate(df_mapping, prompt,temp=0, start=0)


0
Are the following two products the same?
Product 1: Title: sony psp media manager
Description: the psp media manager lets you take advantage of all the extra power held in your psp. it's far more than just a game system -- the psp is a digital photo album movie player and portable music device all rolled into one. psp media manager is the easi
Product 2: Title: sony media manager for psp (98598)
Description: key features: share digital content transfer photos and video ...

Answer with 'True' or 'False'.
__
__
True.
__
__
1


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_88538/818896051.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


Are the following two products the same?
Product 1: Title: punch 5 in 1 home design
Description: 5 in 1 home design is a fully integrated 3-d home design product that includes ultimate deck realmodel 3d home landscaping and furniture design software. punch's 3-d photorealistic technology allows users to fly through their entire design while enjo
Product 2: Title: punch software 20100 punch! 5 in 1 home design
Description: total interior and exterior home design with realistic high-resolution 3d graphics! 20100 punch software

Answer with 'True' or 'False'.
__
__
True
__
__
2


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_88538/818896051.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


Are the following two products the same?
Product 1: Title: movies on psp
Description: x-zoom movies on psp is the first commercial video & dvd software for the sony psp. it enables you to quickly convert your dvds or movie files into a format your psp can read and understand. get the best possible picture and audio quality for 10 diff
Product 2: Title: movies on psp
Description: xoom movies on psp enables you to quickly convert your dvds or movie files into a format your psp can read and understand. get the best possible picture and audio quality for 10 different input formats: dvd (s)vcd divx avi wmv real mpeg quicktime ...

Answer with 'True' or 'False'.
__
__
True
__
__
3


...

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_88538/818896051.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


398


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_88538/818896051.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


399


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_88538/818896051.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


In [28]:
df_mapping


,Unnamed: 0,ltable_id,rtable_id,label,prompt1_en_short_temp_0_gpt4-0613,match_prompt_manual_fewshot_1_eval2_temp_0_gpt3.5_0301,prompt1_en_short_temp_0_gpt3.5-0301
0,0,363,2854,True,True,True,True
1,1,1269,1433,True,True,True,True
2,2,63,2129,True,False,False,True
3,3,540,5,True,True,True,False
4,4,1262,192,True,True,True,True
...,...,...,...,...,...,...,...
395,395,517,465,False,False,False,False
396,396,517,750,False,False,False,False
397,397,517,2839,False,False,False,False
398,398,517,984,False,False,False,False


In [29]:
# saved_prompts_df

In [30]:
# write df to csv
df_mapping.to_csv(PRODUCT_MAPPING, index=False)
saved_prompts_df.to_csv(SAVED_PROMPTS_PATH, index=False)
